# Explicit regularization on MNIST-binary datasets

We examine regularizing two of our smoothness measures: `gradient_norm` and `weights_product`.

This is done on the MNIST-binary datasets.

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

ms = pd.read_feather("0326_mnist_binary/measures.feather")
ms2 = pd.read_feather("0329_mnist_binary_gradient_norm/measures.feather")
ms3 = pd.read_feather("0331_mnist_binary_weights_product/measures.feather")

ms = pd.concat([ms, ms2, ms3, ms_finetune], sort=False)
ms = ms.reset_index(drop=True)

print("Removing {} entries".format(sum(ms["gradient_norm_test"].isna())))
ms = ms[~ms["gradient_norm_test"].isna()]
ms["model.weights_product_reg_coef"] = ms["model.weights_product_reg_coef"].fillna(value=0)

smooth.analysis.remove_constant_columns(ms, verbose=True)

ms["kind"] = ms["kind"].fillna("normal")

for val in [1e-5, 1e-4, 1e-3]:
    ms.loc[ms["model.weights_product_reg_coef"] == val, "kind"] = "weights_product_{:.0e}".format(val)
# ms.loc[ms["model.weights_product_reg_coef"] > 0, "kind"] = "weights_product"

for val in [1e-5, 1e-4, 1e-3]:
    ms.loc[ms["model.gradient_norm_reg_coef"] == val, "kind"] = "gradient_norm_{:.0e}".format(val)

ms

In [ ]:
ms1 = ms[ms["model.weights_product_reg_coef"] == 0]

def plot_measure(measure_name):
    grid = sns.relplot(
        data=ms1,
        x="model.hidden_size",
        y=measure_name,
#         hue="kind",
        hue="model.gradient_norm_reg_coef",
        palette=smooth.analysis.make_palette(ms["model.gradient_norm_reg_coef"]),
#         col="kind",
#         hue_order=sorted(ms1["kind"].unique()),
#         alpha=0.7,
        kind="line",
    )
    ax = grid.axes[0][0]
    if "loss" in measure_name:
        ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim((3, 1200))
    plt.show()

for measure in [
    "loss_train", "loss_test",
#     "gradient_norm_train",
    "gradient_norm_test",
    "weights_product",
    "path_length_f_test",
    "path_length_d_test",
]:
    plot_measure(measure)

In [ ]:
ms1 = ms[ms["model.gradient_norm_reg_coef"] == 0]

def plot_measure(measure_name):
    grid = sns.relplot(
        data=ms1,
        x="model.hidden_size",
        y=measure_name,
#         hue="kind",
        hue="model.weights_product_reg_coef",
        palette=smooth.analysis.make_palette(ms["model.weights_product_reg_coef"]),
#         col="kind",
#         hue_order=sorted(ms1["kind"].unique()),
#         alpha=0.7,
        kind="line",
    )
    ax = grid.axes[0][0]
    if "loss" in measure_name:
        ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim((3, 1200))
    plt.show()

for measure in [
    "loss_train", "loss_test",
    "gradient_norm_test",
    "weights_product",
    "path_length_f_test",
    "path_length_d_test",
]:
    plot_measure(measure)

In [ ]:
ms2 = ms
ms2["log_loss_train"] = np.log10(ms["loss_train"])
ms2["log_loss_test"] = np.log10(ms["loss_test"])

sns.boxplot(
    data=ms,
    x="log_loss_test",
    y="dataset.name",
)

In [ ]:
dataset_stats.sort_values("loss_test")

In [ ]:
sns.relplot(data=dataset_stats, x="loss_train", y="loss_test")